In [1]:
!pip install xgboost
import xgboost as xgb

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import Counter

In [3]:
# Import dataset
cr = pd.read_csv('./data/cs-df1.csv')
cr.shape

(149999, 19)

In [4]:
print(cr.columns)
cr.head()

Index(['seriousdlqin2yrs', 'revolvingutilizationofunsecuredlines', 'age',
       'numberoftime3059dayspastduenotworse', 'debtratio', 'monthlyincome',
       'numberofopencreditlinesandloans', 'numberoftimes90dayslate',
       'numberrealestateloansorlines', 'numberoftime6089dayspastduenotworse',
       'numberofdependents', 'age_group', 'category_pastdue',
       'dependents_groups', 'rsll_groups', 'ocll_quantile_groups',
       'sqrt_debtratio', 'sqrt_monthlyincome',
       'sqrt_revolvingutilizationofunsecuredlines'],
      dtype='object')


,seriousdlqin2yrs,revolvingutilizationofunsecuredlines,age,numberoftime3059dayspastduenotworse,debtratio,monthlyincome,numberofopencreditlinesandloans,numberoftimes90dayslate,numberrealestateloansorlines,numberoftime6089dayspastduenotworse,numberofdependents,age_group,category_pastdue,dependents_groups,rsll_groups,ocll_quantile_groups,sqrt_debtratio,sqrt_monthlyincome,sqrt_revolvingutilizationofunsecuredlines
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2,3,Mostly 30-59 Days Past Due,2,3,3,0.896093,95.498691,0.875287
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1,3,No Past Due,1,0,0,0.349108,50.990195,0.978341
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0,2,Mostly 90 days/+ Past Due,0,0,0,0.291742,55.154329,0.811283
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0,2,No Past Due,0,0,1,0.189868,57.445626,0.483539
4,0,0.907239,49,1,0.024926,19022.0,7,0,1,0,0,3,Mostly 30-59 Days Past Due,0,1,1,0.157879,137.920267,0.952491


In [5]:
X = cr.iloc[:,1:]
X.dtypes

revolvingutilizationofunsecuredlines         float64
age                                            int64
numberoftime3059dayspastduenotworse            int64
debtratio                                    float64
monthlyincome                                float64
numberofopencreditlinesandloans                int64
numberoftimes90dayslate                        int64
numberrealestateloansorlines                   int64
numberoftime6089dayspastduenotworse            int64
numberofdependents                             int64
age_group                                      int64
category_pastdue                              object
dependents_groups                              int64
rsll_groups                                    int64
ocll_quantile_groups                           int64
sqrt_debtratio                               float64
sqrt_monthlyincome                           float64
sqrt_revolvingutilizationofunsecuredlines    float64
dtype: object

In [115]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
# X['age_group'] = lbl.fit_transform(X['age_group'].astype('str'))
X['category_pastdue'] = lbl.fit_transform(X['category_pastdue'].astype('str'))
# X['dependents_groups'] = lbl.fit_transform(X['dependents_groups'].astype('str'))
# X['rsll_groups'] = lbl.fit_transform(X['rsll_groups'].astype('str'))
# X['ocll_quantile_groups'] = lbl.fit_transform(X['ocll_quantile_groups'].astype('str'))
print(X.dtypes)
y = cr[['seriousdlqin2yrs']]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=3011)

revolvingutilizationofunsecuredlines         float64
age                                            int64
numberoftime3059dayspastduenotworse            int64
debtratio                                    float64
monthlyincome                                float64
numberofopencreditlinesandloans                int64
numberoftimes90dayslate                        int64
numberrealestateloansorlines                   int64
numberoftime6089dayspastduenotworse            int64
numberofdependents                             int64
age_group                                      int64
category_pastdue                               int64
dependents_groups                              int64
rsll_groups                                    int64
ocll_quantile_groups                           int64
sqrt_debtratio                               float64
sqrt_monthlyincome                           float64
sqrt_revolvingutilizationofunsecuredlines    float64
dtype: object


In [116]:
X_train.head()

,revolvingutilizationofunsecuredlines,age,numberoftime3059dayspastduenotworse,debtratio,monthlyincome,numberofopencreditlinesandloans,numberoftimes90dayslate,numberrealestateloansorlines,numberoftime6089dayspastduenotworse,numberofdependents,age_group,category_pastdue,dependents_groups,rsll_groups,ocll_quantile_groups,sqrt_debtratio,sqrt_monthlyincome,sqrt_revolvingutilizationofunsecuredlines
141705,0.070246,62,0,1.906765,866.889639,8,0,0,0,0,5,3,0,0,2,1.380856,29.442990,0.265039
42749,0.000000,62,0,0.344276,6000.000000,9,0,2,0,0,5,3,0,2,2,0.586750,77.459667,0.000000
101628,1.017964,23,0,0.006815,2200.000000,1,0,0,0,0,1,3,0,0,0,0.082554,46.904158,1.008942
89010,0.245583,74,0,0.254165,2100.000000,12,0,0,0,0,5,3,0,0,3,0.504147,45.825757,0.495564
86197,0.112231,44,1,0.259409,8900.000000,9,0,1,0,2,3,0,2,1,2,0.509322,94.339811,0.335010


In [117]:
# Original majority/minority ratio
print(Counter(y_train["seriousdlqin2yrs"]))

Counter({0: 111980, 1: 8019})


In [9]:
%pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


# RandomOverSampler

In [61]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy=0.3)

In [62]:
X_train, y_train = oversample.fit_resample(X_train, y_train)
from collections import Counter
print(Counter(y_train["seriousdlqin2yrs"]))

Counter({0: 111980, 1: 33594})


# SMOTE

In [118]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(sampling_strategy=0.5)
X_train, y_train = oversample.fit_resample(X_train, y_train)
print(Counter(y_train["seriousdlqin2yrs"]))

Counter({0: 111980, 1: 55990})


# Up & Down sampling

In [33]:
from sklearn.utils import resample
majority = X_train[y_train==0]
minority = X_train[y_train==1]

In [40]:
majority_down = resample(majority,n_samples=100000,replace=False,random_state=3011)
minority_up = resample(minority,n_samples=50000,replace=True,random_state=3011)
up_down_combined = pd.concat([majority_down,minority_up])
y_majority_down = pd.Series([0] * len(majority_down))
y_minority_up = pd.Series([1] * len(minority_up))
y_up_down_combined = pd.concat([y_majority_down,y_minority_up])

In [41]:
X_train = up_down_combined
y_train = y_up_down_combined


In [44]:
y_up_down_combined.value_counts()

0    100000
1     50000
Name: count, dtype: int64

In [ ]:
# from sklearn.model_selection import GridSearchCV
# para = {
#     'max_depth' : [3,4,5,6,7,8,9,10],
#     'learning_rate' : [0.01,0.1],
#     'n_estimators' : range(80,200,10),
#     'objective' : ['binary:logistic'],  
#     'subsample' : np.linspace(0.01,2,20),
#     'colsample_bytree' : np.linspace(0.5,1,10),
#     'random_state' : [3011]
# }
# model = xgb.XGBClassifier()
# grid_search = GridSearchCV(estimator=model,param_grid=para,cv=5)
# grid_search.fit(X_train,y_train)


In [ ]:
# grid_search.best_score_
# grid_search.best_params_

In [ ]:
# bestxgb_grid = grid_search.best_estimator_.fit(X_train,y_train)

In [ ]:
# from sklearn.metrics import roc_auc_score
# y_predxgb_grid = bestxgb_grid.predict_proba(X_test)[:,1]
# roc_auc_score(y_test,y_predxgb_grid,average='macro', sample_weight=None)

# Train model to find the best parameters

In [119]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV

param_dist = {"n_estimators":sp_randint(100,300),
               "colsample_bytree":np.array([0.5,0.6,0.7,0.8,0.9,1]),
               "subsample":np.array([0.5,0.6,0.7,0.8,0.9,1]),
               "reg_lambda":np.array([1e-5,1e-4,1e-3,1e-2,0.1,1,10,100]),
               "reg_alpha":np.array([1e-5,1e-4,1e-3,1e-2,0.1,1,10,100]),
               "min_child_samples": sp_randint(25,65),
               "min_child_weight": [1],
               "max_depth": sp_randint(1,20),
               "scale_pos_weight":[1],
               "seed" : [3011]}

model = xgb.XGBClassifier(boosting_type = "gbdt",n_jobs =-1,random_state = 0,verbosity =0,scale_pos_weight = 3.2158)
xgb_random_cv_model = RandomizedSearchCV(model, param_distributions=param_dist,
                                    n_iter=20,cv=5,scoring='roc_auc',random_state=42,verbose=1)

# Fit the RandomizedSearchCV to the data
xgb_random_cv_model.fit(X_train, y_train)



Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           boosting_type='gbdt', callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constrain...
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f517294e050>,
                                        'reg_alpha': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                                        'reg_lambda': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                                        'scale_pos_weight': [1], 'seed': [3011],
                                        'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
                   random_state=42, scoring='roc_auc', verbose=1)

In [120]:
xgb_random_cv_model.best_score_

0.9752533258768017

In [121]:
xgb_random_cv_model.best_params_

{'colsample_bytree': 0.8,
 'max_depth': 16,
 'min_child_samples': 39,
 'min_child_weight': 1,
 'n_estimators': 289,
 'reg_alpha': 1.0,
 'reg_lambda': 10.0,
 'scale_pos_weight': 1,
 'seed': 3011,
 'subsample': 1.0}

In [122]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(**xgb_random_cv_model.best_params_)

# Adding bagging 

In [123]:
from sklearn.ensemble import BaggingClassifier

In [124]:
bagging = BaggingClassifier(estimator=xgb_model, n_estimators=10, random_state=3011)

In [125]:
bagging_xgb = bagging.fit(X_train,y_train)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:782: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:41:33] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_child_samples" } are not used.

  warnings.warn(smsg, UserWarning)
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:41:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_child_samples" } are not used.

  warnings.warn(smsg, UserWarning)
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:41:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_child_samples" } are not used.

  warnings.warn(smsg, UserWarning)
/home/codespace/.python/cu

In [126]:
bagging_xgb.score(X_test,y_test)

0.9243333333333333

In [127]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
y_pred_prob = bagging_xgb.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test,y_pred_prob,average='macro', sample_weight=None)

In [128]:
y_pred = bagging_xgb.predict(X_test)
print(y_pred)

[0 1 0 ... 0 0 0]


In [129]:
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)

In [130]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("ROC-AUC:", roc_auc)

Accuracy: 0.9243333333333333
Precision: 0.4171392564587272
Recall: 0.32984554060787247
ROC-AUC: 0.8397431765942055
